todo:

- least squares?

This notebook assumes you have a basic knowledge of matrix algebra that you get from A-Level maths.

## Gaussian Operations

A linear system is a set of linear equations, often $n$ equations in terms of $n$ variables.

$$
\begin{alignat}{3}
       & -7y && -4z && =2  \\
    2x & +4y && +6z && =12 \\
    3x & +y  && -z  && =-2
\end{alignat}
$$

It's important to be able to solve for $x$, $y$ and $z$ which satisfy all of these equations. To do this, we're going to put the coefficients and constants here into a matrix which we can manipulate.

$$
\left[\begin{array}{ccc|c}
    0 & -7 & -4 & 2  \\
    2 &  4 &  6 & 12 \\
    3 &  1 & -1 & -2
\end{array}\right]
$$

This is called the *augmented* matrix of the linear system. The vertical bar is just cosmetic.

There are three kinds of operations we can perform on this matrix without changing the solution to the linear system it represents:

1. multiplying a row by a scalar value
2. swapping two rows
3. adding a multiple of one row to another

These are called *Gaussian operations*.

In [173]:
⍝ augmented matrix
m←↑(0 ¯7 ¯4 2)(2 4 6 12)(3 1 ¯1 ¯2)

m[1 2;]←m[2 1;]  ⍝ swap row 1 and row 2
m[1;]÷←2         ⍝ scale row 1 by 0.5
m[3;]+←¯3×m[1;]  ⍝ add ¯3 times row 1 to row 3
m

1 2 3 6
0 ¯7 ¯4 2
0 ¯5 ¯10 ¯20

A matrix is in *row echelon form* when the first nonzero element of each row is to the right of the first nonzero element in the row above. It is in *reduced* row echelon form when the first nonzero element of each row is a $1$. We can use Gaussian operations to reduce an augmented matrix to reduced row echelon form (RREF).

In [172]:
m[2 3;]←m[3 2;]
m[2;]÷←¯5
m[3;]+←7×m[2;]
m[3;]÷←10
m[2;]-←2×m[3;]
m[1;]-←3×m[3;]
m[1;]-←2×m[2;]
m

1 0 0 1
0 1 0 ¯2
0 0 1 3

For systems with a single solution, this form shows us exactly that solution! We just convert back to the linear system form.

$$
\left[\begin{array}{ccc|c}
    1 & 0 & 0 & 1  \\
    0 & 1 & 0 & -2 \\
    0 & 0 & 1 & 3
\end{array}\right]
\leftrightarrow
\begin{alignat}{1}
    x & = 1  \\
    y & = -2 \\
    z & = 3
\end{alignat}
$$

We can automate this process:

In [174]:
    ∇ a←Rref a;⎕IO;i;j;m;n;nonzero
      ⎕IO←1
      j←0                                 ⍝ column we're considering
      (m n)←⍴a
      :For i :In ⍳m                       ⍝ for each row i
          :Repeat                         ⍝ find next column j with a nonzero below
              j+←1
              :If j>n ⋄ :Return ⋄ :EndIf  ⍝ early return if we run out of columns
              nonzero←⍸0≠0@(⍳i-1)⊢a[;j]   ⍝ where are the nonzeros in row i or below
          :Until nonzero≢⍬
          a←⊖@(i,⊃nonzero)⊢a              ⍝ swap row up if necessary (leftmost nonzero needs to be higher)
          a[i;]÷←a[i;j]                   ⍝ scale row so leading variable is a one
          a+←a[i;]×⍤1 0⊢0@i-a[;j]         ⍝ zero out rest of column by adding multiples of this row
      :EndFor
    ∇

The same process can be used to find the inverse of a matrix (if it exists). The inverse of a square matrix $A$ can be found by gluing the identity matrix to it's right, and then finding the RREF of that matrix. If the RREF has the identity matrix as the left half after this transformation, then the right half will be $A^{-1}$.

In [191]:
Id←,⍨⍴1↑⍨1+⊢
m←↑(2 1)(1 1) ⍝ initial matrix
m
a←m,Id 2      ⍝ glue identity to the right
a
⍝ find RREF
a[1;]-←a[2;]
a[2;]-←a[1;]
a
⍝ the righthand side is the inverse if the lefthand side is the identity
(Id 2)≡2∘↑⍤1⊢a
minv←2∘↓⍤1⊢a
minv
⍝ we can check that this is indeed the inverse
(Id 2)≡minv+.×m

2 1
1 1

2 1 1 0
1 1 0 1

1 0 1 ¯1
0 1 ¯1 2

1

1 ¯1
¯1 2

1

This can be automated as well:

In [193]:
]dinput
Invert←{
    n←⊃⍴⍵
    id←Id n
    rref←Rref⍵,id
    id≡n↑⍤1⊢rref: n↓⍤1⊢rref
    ⎕SIGNAL 11 ⍝ domain error if left side is not the indentiy
}

In [196]:
Invert m

1 ¯1
¯1 2

Of course this is provided as a primitive.

In [197]:
⌹m

1 ¯1
¯1 2

## Matrices as Transformations

In some vector $\left[\begin{matrix}x\\y\end{matrix}\right]$, $x$ represents the scale factor of the $x$ coordinate vector $\left[\begin{matrix}1\\0\end{matrix}\right]$, and likewise $y$ represents the scale factor of $\left[\begin{matrix}0\\1\end{matrix}\right]$.

$$
\left[\begin{matrix}x\\y\end{matrix}\right]
=
x\left[\begin{matrix}1\\0\end{matrix}\right]
+
y\left[\begin{matrix}0\\1\end{matrix}\right]
$$

When we do matrix multiplication of a vector we get:

$$
\left[\begin{matrix}a&b\\c&d\end{matrix}\right]
\left[\begin{matrix}x\\y\end{matrix}\right]
=
\left[\begin{matrix}ax+by\\cx+dy\end{matrix}\right]
=
x\left[\begin{matrix}a\\c\end{matrix}\right]
+
y\left[\begin{matrix}b\\d\end{matrix}\right]
$$

That final form looks just like our decomposition of $\left[\begin{matrix}x\\y\end{matrix}\right]$, except with the column vectors of our matrix $\left[\begin{matrix}a\\c\end{matrix}\right]$ and $\left[\begin{matrix}b\\d\end{matrix}\right]$ instead of the standard coordinates. This tells us what matrix multiplication means geometrically. The columns of the matrix describe where the standard coordinates are translated to, and since all vectors are just linear combinations of the standard coordinates, that tells us how all vectors are transformed.

This is why the identity matrix $\left[\begin{matrix}1&0\\0&1\end{matrix}\right]$ has no effect in matrix multiplication. The standard coordinates are translated to themselves, unchanged.

This lets us easily define matrices for certain transformations. For example $\left[\begin{matrix}1&0\\0&2\end{matrix}\right]$ to stretch vertically, or $\left[\begin{matrix}¯1&0\\0&1\end{matrix}\right]$ to reflect horizontally.

We're going to define a helper function to visualise this.

In [117]:
]dinput
PlotPoints←{(spec points)←⍺ ⍵ ⋄ ⎕IO←0
    ((minx maxx)(miny maxy)(resx resy))←spec,(3>≢spec)↑⊂1 1
    xlen←⌊resx÷⍨maxx-minx
    ylen←⌊resy÷⍨maxy-miny
    xaxis←minx+resx×⍳xlen
    yaxis←miny+resy×⍳ylen
    plot←ylen xlen⍴' '
    y0←yaxis⍸0
    x0←xaxis⍸0
    plot←'─'@y0⊢plot
    plot←'│'@x0⍤1⊢plot
    plot[y0;x0]←'0'
    _←{(x y)←,⍵ ⋄ plot[yaxis⍸y;xaxis⍸x]←'*'}¨points
    ⊖plot
}

In [153]:
logo←(⍪¯2 1)(⍪¯2 2)(⍪¯1 2)(⍪0 2)(⍪1 2)(⍪2 1)(⍪2 0)(⍪2 ¯1)(⍪1 ¯2)(⍪0 ¯2)(⍪¯1 ¯2)(⍪¯2 ¯2)
Plot←(¯5 5)(¯5 5)(1 1)∘PlotPoints

Plot logo                ⍝ original image
stretch←↑(1 0)(0 2)      ⍝ stretch by factor 2 vertically (notice it's the y column that has changed)
Plot stretch∘(+.×)¨logo
reflect←↑(¯1 0)(0 1)     ⍝ reflect in y axis, all x coords are negated
Plot reflect∘(+.×)¨logo
shear←↑(1 1)(0 1)        ⍝ a shear that's difficult to describe in words so just look at it
Plot shear∘(+.×)¨logo

│ 
 │ 
 **** 
 * │ * 
─────0─*──
 │ * 
 **** 
 │ 
 │ 
 │

**** 
 │ 
 * │ * 
 │ 
─────0─*──
 │ 
 │ * 
 │ 
 **** 
 │

│ 
 │ 
 **** 
 * │ * 
───*─0────
 * │ 
 **** 
 │ 
 │ 
 │

│ 
 │ 
 **** 
 *│ * 
─────0─*──
 │* 
 ****│ 
 │ 
 │ 
 │

Multiplying two matrices corresponds to composing the transformations they represent. This is easy to see when you consider the associativity of matrix multiplication: $(AB)v=A(Bv)$. Multiplying by the product of $A$ and $B$ is the same as first multiplying by $B$, then by $A$. This is helps us understand why matrix multiplication is not commutative, the order of transformations affects the result. For example, a shear followed by a reflection is not the same as a reflection followed by a shear.

In [195]:
Plot (reflect+.×shear+.×⊢)¨logo ⍝ shear, then reflect
Plot (shear+.×reflect+.×⊢)¨logo ⍝ reflect, then shear

│ 
 │ 
 **** 
 * │* 
───*─0────
 *│ 
 │****
 │ 
 │ 
 │

│ 
 │ 
 │****
 *│ * 
───*─0────
 * │ 
 **** 
 │ 
 │ 
 │

As you might guess, the inverse of a matrix performs the inverse of a transformation.

In [199]:
Plot logo
sheared←(shear+.×⊢)¨logo
Plot sheared
unsheared←((⌹shear)+.×⊢)¨sheared
logo≡unsheared
Plot unsheared

│ 
 │ 
 **** 
 * │ * 
─────0─*──
 │ * 
 **** 
 │ 
 │ 
 │

│ 
 │ 
 **** 
 *│ * 
─────0─*──
 │* 
 ****│ 
 │ 
 │ 
 │

1

│ 
 │ 
 **** 
 * │ * 
─────0─*──
 │ * 
 **** 
 │ 
 │ 
 │

Some matrices will take all of 2d space and squash it onto a line:

In [130]:
squash←↑(1 ¯1)(1 ¯1)
Plot squash∘(+.×)¨logo

┌→─────────┐
↓ │ │
│ │ * │
│ │ * │
│ │* │
│─────*────│
│ *│ │
│ * │ │
│ * │ │
│ * │ │
│ │ │
└──────────┘

In this case, we've 'lost a dimension'. We can always tell when a matrix will do this by looking at its reduced row echelon form.

In [131]:
Rref shear
Rref squash

┌→──┐
↓1 0│
│0 1│
└~──┘

┌→───┐
↓1 ¯1│
│0 0│
└~───┘

What can we notice? The RREF of the shear has two pivot columns (recall, these are columns with all zeros except a single one), while the RREF of the squash has only one pivot column. This is exactly the pattern - the number of pivot columns is the number of dimensions of the result of a matrix's transformation. We call this the matrix's *rank*. Likewise, the number of pivot columns is the number of dimensions you lose. We call this it's *nullity*.

In [134]:
⍝ bitmask of which columns are pivots
⍝       ┌─────────────┬─sum is 1 (there's one 1) and ⎕dr is 11, meaning boolean, meaning only 1s and 0s
Pivots←{(∧/1 11=+/,⎕dr)⍤1⍉Rref⍵}

Rank←+/Pivots      ⍝ number of pivots
Nullity←+/⍤~Pivots ⍝ number of not-pivots

In [133]:
Rank shear
Nullity shear
Rank squash
Nullity squash

2

0

1

1

## Determinants

Transformations will often scale things. The *determinant* of a matrix gives you a measure of this. For a $2\times 2$ matrix $\left[\begin{matrix}a&b\\c&d\end{matrix}\right]$, the determinant is $ad-bc$.

In [161]:
Det2x2←{-/×⌿0 1⌽⍵}

So for example, the matrix $\left[\begin{matrix}1&0\\0&2\end{matrix}\right]$ from earlier scales by $2$ in the $y$ direction, so the determinant is $2$.

In [163]:
Plot logo
Plot (stretch+.×⊢)¨logo
Det2x2 stretch

│ 
 │ 
 **** 
 * │ * 
─────0─*──
 │ * 
 **** 
 │ 
 │ 
 │

**** 
 │ 
 * │ * 
 │ 
─────0─*──
 │ 
 │ * 
 │ 
 **** 
 │

2

When a matrix has a nonzero nullity, it would take an area and compress it down to a lower dimensional space, something with no area, so the determinant is $0$.

In [165]:
Plot logo
Plot (squash+.×⊢)¨logo
Det2x2 squash

│ 
 │ 
 **** 
 * │ * 
─────0─*──
 │ * 
 **** 
 │ 
 │ 
 │

│ 
 │ * 
 │ * 
 │* 
─────*────
 *│ 
 * │ 
 * │ 
 * │ 
 │

0

For higher dimensional matrices, we need a more complicated procedure to find the determinant $\text{det}(A)$ of a $n\times n$ matrix $A$.

Let $A_{ij}$ be the $n-1\times n-1$ *minor* matrix found by deleting row $i$ and column $j$ from $A$.

And let $C_{ij}=(-1)^{i+j}\text{det}(A_{ij})$ be the *cofactor* of $i,j$.

Then the determinant can be found either by fixing an $i$ and calculating

$$\text{det}(A)=\sum_{j=1}^na_{ij}C_{ij}$$

or by fixing a $j$ and finding

$$\text{det}(A)=\sum_{i=1}^na_{ij}C_{ij}$$

Deriving these nasty formulas is out of scope. But we can easily implement them.

In [169]:
]dinput
Det←{⎕IO←1
    a←⍵
    n←≢a
    n=1: ⊃a
    ⍝ expand cofactors along 1st row (i=1)
    ⍝        ┌cofactors──────────────────┐
    ⍝        │┌¯1^i+j┐     ┌minor───┐    │ 
    +/(,1↑a)×((¯1*1+⊢)×{Det(⍵≠⍳n)/1↓a}¨)⍳n    
}

In [170]:
Det↑(1 0)(0 1)
Det↑(2 5 ¯3 ¯2)(¯2 ¯3 2 ¯5)(1 3 ¯2 0)(¯1 6 4 0)

1

¯103

## Change of Basis

We've seen how matrices take the coordinates of a vector and put them into a new coordinate system. Sometimes, we want to not actually change a vector, but just find what its coordinates would be a different coordinate system. Formally this is called *change of basis* where a *basis* is a coordinate system. The *standard basis* is the normal coordinates $\left[\begin{matrix}1\\0\end{matrix}\right]$ and $\left[\begin{matrix}0\\1\end{matrix}\right]$.

So to find the representation of some vector $\left[\begin{matrix}x\\y\end{matrix}\right]$ in a new basis $\left[\begin{matrix}a\\c\end{matrix}\right],\left[\begin{matrix}b\\d\end{matrix}\right]$, we want to find $x',y'$ such that

$$
\begin{align}
    x'\left[\begin{matrix}a\\c\end{matrix}\right]
    +
    y'\left[\begin{matrix}b\\d\end{matrix}\right]
    &=
    \left[\begin{matrix}x\\y\end{matrix}\right]
    \\
    \left[\begin{matrix}a&b\\c&d\end{matrix}\right]
    \left[\begin{matrix}x'\\y'\end{matrix}\right]
    &=
    \left[\begin{matrix}x\\y\end{matrix}\right]
    \\
    \left[\begin{matrix}x'\\y'\end{matrix}\right]
    &=
    \left[\begin{matrix}a&b\\c&d\end{matrix}\right]^{-1}
    \left[\begin{matrix}x\\y\end{matrix}\right]
\end{align}
$$

Neatly enough, this doesn't make the assumption that $\left[\begin{matrix}x\\y\end{matrix}\right]$, $\left[\begin{matrix}a\\c\end{matrix}\right]$, and $\left[\begin{matrix}b\\d\end{matrix}\right]$ are in the standard basis, only that they are all in the same basis, so this method can be used to convert between any basis!

In [166]:
⍝┌─────────┬─new basis
⍝│         │   ┌──┬─old vector
(↑(0 1)(1 0))⌹⍨⍪3 4


4
3

Since matrices are just collections of column vectors, we can use exactly the same method to change their bases.

In [144]:
b←↑(2 1)(1 0)      ⍝ basis b
d←↑(¯1 1)(1 1)     ⍝ basis d
bd←d⌹⍨b            ⍝ change of basis b → d

`bd` has as its columns the basis for `b` represented in the basis `d`. Therefore, multiplying by it will take a vector whose coordinates are in `b` and return the same vector but with coordinates in basis `d`.

In [146]:
vb←⍪0 ¯1           ⍝ vector in basis b
b+.×vb             ⍝ vb → standard basis
d+.×bd+.×vb        ⍝ vb → d → standard basis

¯1
 0

¯1
 0

We can use matrices like this to take a transformation which works in one basis, and run it on vectors in another basis. For example, if we have many points in basis `b`, but we want to transform them in basis `d`, for example by reflecting them in `d`'s first basis (it's analagoy of $x$).

In [200]:
⍝ we intially made logo in the standard basis, so for this example lets put them into basis b
logob←(b⌹⍨⊢)¨logo
⍝          ┌──────┬─convert back d → b
⍝          │      │┌────────┬─reflect in basis d
⍝          │      ││        │┌───┬─change basis b → d
reflected←((⌹bd)+.×reflect+.×bd+.×⊢)¨logob

⍝   ┌──┬─round out floating point errors
⍝   │  │┌──────┬─convert back to standard basis so we can plot
Plot⌊.5+(b+.×⊢)¨reflected

│ 
 │ 
 *** 
 * │ * 
───*─0─*──
 * │ * 
 * │** 
 │ 
 │ 
 │

The matrix we ultimately used to transform our points was `(⌹bd)+.×reflect+.×bd`. When two matrices $A$ and $B$ represent the same transformation but in different bases, and we can convert between those bases with some change-of-basis matrix $C$ (so $A=C^{-1}BC$), we say those matrices are *similar*. So in our case, `(⌹bd)+.×reflect+.×bd` is *similar* to `reflect`.

## eigenvalues and eigenvectors

In [79]:
Trace←{⎕io←1 ⋄ +/1 1⍉⍵}

In [82]:
Trace↑(1 0)(0 1)

2